## Test

In [1]:
# Import

import epics
import numpy
import pandas
import torch
import nufft

import sys
sys.path.append('..')

from harmonica.window import Window
from harmonica.data import Data
from harmonica.frequency import Frequency

torch.set_printoptions(precision=12, sci_mode=True)
torch.cuda.is_available()

True

In [2]:
!python --version

Python 3.10.10


In [3]:
torch.__version__

'2.0.1+cu117'

In [4]:
size = 1024
length = 8192
dtype = torch.float64

In [5]:
# Test data (cpu)
device = 'cpu'
w = Window(length, 'cosine_window', 1.0, dtype=dtype, device=device)
data = torch.sin(2*numpy.pi*0.12*torch.linspace(1, len(w), len(w), dtype=w.dtype, device=w.device))
d = Data.from_data(w, torch.stack([data for _ in range(size)]))
f = Frequency(d)

In [6]:
%%time
f('parabola')

CPU times: user 1.48 s, sys: 483 ms, total: 1.96 s
Wall time: 200 ms


In [7]:
# Clear
del w, d, f

In [8]:
# Test data (gpu)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
w = Window(length, 'cosine_window', 1.0, dtype=dtype, device=device)
data = torch.sin(2*numpy.pi*0.12*torch.linspace(1, len(w), len(w), dtype=w.dtype, device=w.device))
d = Data.from_data(w, torch.stack([data for _ in range(size)]))
f = Frequency(d)

In [9]:
%%time
f('parabola')

CPU times: user 361 ms, sys: 5.92 ms, total: 367 ms
Wall time: 366 ms


In [10]:
# Clear
del w, d, f
torch.cuda.synchronize()
torch.cuda.empty_cache()

In [11]:
# SVD

In [12]:
# Test data (cpu)
device = 'cpu'
w = Window(512, 'cosine_window', 1.0, dtype=dtype, device=device)
data = torch.sin(2*numpy.pi*0.12*torch.linspace(1, len(w), len(w), dtype=w.dtype, device=w.device))
data = torch.stack([data for _ in range(512)])
data = torch.stack([data for _ in range(64)])
print(data.shape)
print(data.device)

torch.Size([64, 512, 512])
cpu


In [13]:
%%time
u, s, v = torch.linalg.svd(data)

CPU times: user 21.7 s, sys: 4.25 s, total: 26 s
Wall time: 8.14 s


In [14]:
# Clear
del w, data

In [15]:
# Test data (gpu)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
w = Window(512, 'cosine_window', 1.0, dtype=dtype, device=device)
data = torch.sin(2*numpy.pi*0.12*torch.linspace(1, len(w), len(w), dtype=w.dtype, device=w.device))
data = torch.stack([data for _ in range(512)])
data = torch.stack([data for _ in range(64)])
print(data.shape)
print(data.device)

torch.Size([64, 512, 512])
cuda:0


In [16]:
%%time
u, s, v = torch.linalg.svd(data)

CPU times: user 9.98 s, sys: 244 ms, total: 10.2 s
Wall time: 10.2 s


In [17]:
# Clear
del w, data
torch.cuda.synchronize()
torch.cuda.empty_cache()

In [18]:
# PV

In [19]:
w = Window(4096, 'cosine_window', 1.0, dtype=torch.float64, device='cpu')
pv_list = ["H:STP2:DATA:X", "H:STP4:DATA:X", "H:SRP1:DATA:X", "H:SRP2:DATA:X", "H:SRP3:DATA:X", "H:SRP4:DATA:X", "H:SRP5:DATA:X", "H:SRP6:DATA:X", "H:SRP7:DATA:X", "H:SRP8:DATA:X", "H:SRP9:DATA:X", "H:SIP1:DATA:X", "H:SIP2:DATA:X", "H:SRP10:DATA:X", "H:SRP11:DATA:X", "H:SRP12:DATA:X", "H:SRP13:DATA:X", "H:SRP14:DATA:X", "H:SRP15:DATA:X", "H:SRP16:DATA:X", "H:SRP17:DATA:X", "H:SEP5:DATA:X", "H:SEP4:DATA:X", "H:SEP3:DATA:X", "H:SEP1:DATA:X", "H:SEP0:DATA:X", "H:NEP0:DATA:X", "H:NEP1:DATA:X", "H:NEP3:DATA:X", "H:NEP4:DATA:X", "H:NEP5:DATA:X", "H:NRP17:DATA:X", "H:NRP16:DATA:X", "H:NRP15:DATA:X", "H:NRP14:DATA:X", "H:NRP13:DATA:X", "H:NRP12:DATA:X", "H:NRP11:DATA:X", "H:NRP10:DATA:X", "H:NIP3:DATA:X", "H:NIP1:DATA:X", "H:NRP9:DATA:X", "H:NRP8:DATA:X", "H:NRP7:DATA:X", "H:NRP6:DATA:X", "H:NRP5:DATA:X", "H:NRP4:DATA:X", "H:NRP3:DATA:X", "H:NRP2:DATA:X", "H:NRP1:DATA:X", "H:NTP4:DATA:X", "H:NTP2:DATA:X", "H:NTP0:DATA:X", "H:STP0:DATA:X"]
pv_rise = [0 for _ in range(len(pv_list))]

In [20]:
%%time
d = Data.from_epics(w, pv_list, pv_rise=pv_rise)

CPU times: user 857 µs, sys: 26.5 ms, total: 27.3 ms
Wall time: 41.9 ms


In [21]:
!nvidia-smi

Fri Aug  4 12:09:32 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.125.06   Driver Version: 525.125.06   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0  On |                  N/A |
|ERR!   52C    P3    30W / 240W |   1954MiB /  8192MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------